In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
lr = 1e-3
env = gym.make('CartPole-v0')
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000

[2017-09-02 15:25:26,995] Making new env: CartPole-v0


In [3]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break
                
# some_random_games_first()

In [8]:
# To get training data
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                    
                elif data[1] == 0:
                    output = [1, 0]
                    
                training_data.append([data[0], output])
                
        env.reset()
        scores.append(score)
    training_data_save = np.array(training_data)
    np.save('saved.npy', training_data_save)
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    return training_data

In [9]:
initial_population()

Average accepted score:  61.83195592286501
Median accepted score:  58.0
Counter({52.0: 34, 51.0: 27, 54.0: 24, 50.0: 22, 56.0: 20, 58.0: 18, 57.0: 17, 55.0: 15, 61.0: 14, 53.0: 14, 59.0: 14, 62.0: 13, 63.0: 11, 65.0: 10, 70.0: 9, 60.0: 9, 69.0: 9, 68.0: 8, 66.0: 7, 64.0: 7, 76.0: 5, 84.0: 4, 78.0: 4, 75.0: 4, 82.0: 4, 74.0: 4, 79.0: 3, 71.0: 3, 67.0: 3, 77.0: 3, 96.0: 2, 86.0: 2, 73.0: 2, 85.0: 1, 107.0: 1, 81.0: 1, 92.0: 1, 131.0: 1, 121.0: 1, 112.0: 1, 100.0: 1, 104.0: 1, 87.0: 1, 83.0: 1, 99.0: 1, 95.0: 1, 105.0: 1, 80.0: 1, 88.0: 1, 102.0: 1, 114.0: 1})


[[array([ 0.04631577,  0.24118822, -0.0117392 , -0.27928877]), [1, 0]],
 [array([ 0.05113954,  0.04623569, -0.01732497,  0.00966865]), [1, 0]],
 [array([ 0.05206425, -0.14863357, -0.0171316 ,  0.29683538]), [1, 0]],
 [array([ 0.04909158, -0.34350717, -0.01119489,  0.58406644]), [1, 0]],
 [array([  4.22214355e-02,  -5.38470517e-01,   4.86438848e-04,
           8.73201908e-01]), [0, 1]],
 [array([ 0.03145203, -0.34335518,  0.01795048,  0.58067196]), [0, 1]],
 [array([ 0.02458492, -0.1484893 ,  0.02956392,  0.29369738]), [0, 1]],
 [array([ 0.02161514,  0.04619895,  0.03543786,  0.01048315]), [1, 0]],
 [array([ 0.02253911, -0.14941284,  0.03564753,  0.31413315]), [0, 1]],
 [array([ 0.01955086,  0.04518365,  0.04193019,  0.03290201]), [1, 0]],
 [array([ 0.02045453, -0.15051373,  0.04258823,  0.33851394]), [0, 1]],
 [array([ 0.01744426,  0.04397715,  0.04935851,  0.05955931]), [0, 1]],
 [array([ 0.0183238 ,  0.23835792,  0.05054969, -0.21715124]), [1, 0]],
 [array([ 0.02309096,  0.04255115, 

In [10]:
def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network)
    return model

In [12]:
def train_model(training_data, model=False):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]), 1)
    y = [i[1] for i in training_data]
    
    if not model:
        model = neural_network_model(input_size=len(X[0]))
        
    model.fit({'input': X}, {'targets': y}, n_epoch=5, snapshot_step=500, show_metric=True, run_id='openai_cart_pole_v0')
    return model

In [ ]:
training_data = initial_population()
model = train_model(training_data)